In [ ]:
import os
import rasterio
from rasterio.warp import calculate_default_transform, reproject, Resampling
import numpy as np
import pandas as pd
from scipy.stats import pearsonr

# ============================
# Function: Read raster (.tif) as NumPy array
# - Converts to float32
# - Replaces NoData values with NaN
# ============================
def read_tif_as_array(path):
    with rasterio.open(path) as src:
        arr = src.read(1).astype(np.float32)
        nodata = src.nodata
        if nodata is not None:
            arr[arr == nodata] = np.nan
    return arr


# ============================
# Function: Reproject and resample a raster
# to match the reference raster in:
#   - CRS (coordinate system)
#   - resolution
#   - extent
#   - dimensions
# Uses nearest-neighbor resampling
# ============================
def reproject_to_match(src_path, ref_path, dst_path):
    with rasterio.open(ref_path) as ref:
        ref_crs = ref.crs
        ref_transform = ref.transform
        ref_width = ref.width
        ref_height = ref.height

    with rasterio.open(src_path) as src:
        src_array = src.read(1)
        dst_array = np.empty((ref_height, ref_width), dtype=np.float32)

        kwargs = src.meta.copy()
        kwargs.update({
            'crs': ref_crs,
            'transform': ref_transform,
            'width': ref_width,
            'height': ref_height
        })

        with rasterio.open(dst_path, 'w', **kwargs) as dst:
            reproject(
                source=src_array,
                destination=dst_array,
                src_transform=src.transform,
                src_crs=src.crs,
                dst_transform=ref_transform,
                dst_crs=ref_crs,
                resampling=Resampling.nearest
            )
            dst.write(dst_array, 1)


# ============================
# Function: Batch correlation analysis
# Between one ecosystem service raster (es_tif_path)
# and all environmental variable rasters in a folder (env_folder)
# For each variable:
#   1. Align spatial resolution and projection
#   2. Flatten both arrays and remove NaN
#   3. Compute Pearson correlation (r)
#   4. Record number of valid pixels
#   5. Save results to an Excel file
# ============================
def analyze_batch(env_folder, es_tif_path, output_excel="correlation_results.xlsx"):
    es_name = os.path.splitext(os.path.basename(es_tif_path))[0]
    es_array = read_tif_as_array(es_tif_path).flatten()

    results = []
    total_pixels = len(es_array)

    for fname in os.listdir(env_folder):
        if not fname.endswith(".tif"):
            continue

        env_path = os.path.join(env_folder, fname)
        temp_path = os.path.join(env_folder, "_temp_aligned.tif")
        
        # Reproject to match the reference raster
        reproject_to_match(env_path, es_tif_path, temp_path)

        # Read resampled raster data
        env_array = read_tif_as_array(temp_path).flatten()

        # Remove NaN values from both datasets
        df = pd.DataFrame({"env": env_array, "es": es_array}).dropna()

        if df.empty:
            r = np.nan
            valid_pixels = 0
        else:
            # Compute Pearson correlation coefficient
            r, _ = pearsonr(df["env"], df["es"])
            valid_pixels = len(df)

        # Store results
        results.append({
            "Environment_Variable": os.path.splitext(fname)[0],
            "Ecosystem_Service": es_name,
            "Pearson_r": round(r, 3) if not np.isnan(r) else "NA",
            "Valid_Pixels": valid_pixels,
            "Total_Pixels": total_pixels
        })

        # Delete temporary file
        if os.path.exists(temp_path):
            os.remove(temp_path)

    # Save all results to Excel
    df_result = pd.DataFrame(results)
    df_result.to_excel(output_excel, index=False)
    print(f"✅ All analyses completed. Results saved to: {output_excel}")


# ================= Example Paths =================
env_folder = r"D:\Documents\Desktop\ESDR-P\tif_0.005deg"  # Folder containing environmental variable rasters
es_tif_path = r"D:\Documents\Desktop\ESDR-P\tif_0.005deg\CS_ESDR.tif"  # Ecosystem service raster
output_excel = r"D:\Documents\Desktop\ESDR-P\tif_0.005deg\CS_ESDRcorrelation_results.xlsx"  # Output file path

# Run batch correlation analysis
analyze_batch(env_folder, es_tif_path, output_excel)
